# Macaw OpenVoice — CLI Validation Notebook

Validates **all CLI commands** of Macaw OpenVoice.
Run all cells sequentially on Google Colab (GPU runtime recommended).

**What this validates:**
- Package installation (end-user experience)
- Model management: `catalog`, `pull`, `list`, `inspect`, `remove`
- Server management: `serve`, `ps`
- Transcription: all output formats, language, ITN control
- Translation: audio-to-English

> **Pre-requisite:** Select a GPU runtime (Runtime > Change runtime type > T4 GPU)

## 1. Configuration

In [ ]:
# Edit these before running
STT_MODEL = "faster-whisper-tiny"
TTS_MODEL = "kokoro-v1"
SERVER_PORT = 8000
BASE_URL = f"http://localhost:{SERVER_PORT}"

# Installation source:
#   'pip'     — install released version from PyPI
#   'develop' — install from git develop branch (pre-release validation)
INSTALL_FROM = "develop"

## 2. Installation

Installs from **PyPI** or **git develop branch** based on `INSTALL_FROM` above.

> **Note (Colab):** You may see a pip resolver warning about `protobuf` conflicts with pre-installed `tensorflow`/`grpcio-status`. This is harmless — Macaw requires `protobuf>=6.31` and does not use TensorFlow. The warning can be safely ignored.

In [ ]:
%%capture
if INSTALL_FROM == "develop":
    !pip install 'macaw-openvoice[faster-whisper,kokoro,itn,codec] @ git+https://github.com/usemacaw/macaw-openvoice.git@develop'
else:
    !pip install macaw-openvoice[faster-whisper,kokoro,itn,codec]

In [ ]:
!macaw --version

## 3. Model Management

### 3.1 `macaw catalog`

In [ ]:
!macaw catalog

### 3.2 `macaw pull`

In [ ]:
!macaw pull $STT_MODEL

In [ ]:
!macaw pull $TTS_MODEL

### 3.3 `macaw list`

In [ ]:
!macaw list

### 3.4 `macaw inspect`

In [ ]:
!macaw inspect $STT_MODEL

In [ ]:
!macaw inspect $TTS_MODEL

## 4. Server Management

### 4.1 Start server in background

In [ ]:
import os

os.environ["MACAW_WORKER_HEALTH_PROBE_TIMEOUT_S"] = "300"

In [ ]:
!nohup macaw serve --host 0.0.0.0 --port $SERVER_PORT > /tmp/macaw.log 2>&1 &

In [ ]:
import time

import httpx

print("Waiting for server ...")
for i in range(90):
    try:
        r = httpx.get(f"{BASE_URL}/health", timeout=5)
        if r.json().get("status") == "ok":
            print(f"Server ready! (attempt {i + 1})")
            print(r.json())
            break
    except Exception:
        pass
    time.sleep(2)
else:
    print("Server did not start. Logs:")
    !tail -50 /tmp/macaw.log
    raise RuntimeError("Server not ready")

### 4.2 `macaw ps`

In [ ]:
!macaw ps --server $BASE_URL

## 5. Generate Test Audio

Uses TTS to generate speech, then uses that audio for STT testing (round-trip).

In [ ]:
import httpx

TEST_TEXT = "Hello world. This is a test of the Macaw voice system."
r = httpx.post(
    f"{BASE_URL}/v1/audio/speech",
    json={"model": TTS_MODEL, "input": TEST_TEXT, "voice": "default"},
    timeout=120,
)
assert r.status_code == 200, f"TTS failed: {r.status_code} {r.text}"

with open("/tmp/test_audio.wav", "wb") as f:
    f.write(r.content)
print(f"Test audio saved: {len(r.content):,} bytes")

## 6. Transcription Tests

### 6.1 JSON format (default)

In [ ]:
!macaw transcribe /tmp/test_audio.wav -m $STT_MODEL --format json --server $BASE_URL

### 6.2 Text format

In [ ]:
!macaw transcribe /tmp/test_audio.wav -m $STT_MODEL --format text --server $BASE_URL

### 6.3 Verbose JSON format

In [ ]:
!macaw transcribe /tmp/test_audio.wav -m $STT_MODEL --format verbose_json --server $BASE_URL

### 6.4 SRT subtitle format

In [ ]:
!macaw transcribe /tmp/test_audio.wav -m $STT_MODEL --format srt --server $BASE_URL

### 6.5 VTT subtitle format

In [ ]:
!macaw transcribe /tmp/test_audio.wav -m $STT_MODEL --format vtt --server $BASE_URL

### 6.6 With explicit language

In [ ]:
!macaw transcribe /tmp/test_audio.wav -m $STT_MODEL --language en --server $BASE_URL

### 6.7 With ITN disabled

In [ ]:
!macaw transcribe /tmp/test_audio.wav -m $STT_MODEL --no-itn --server $BASE_URL

## 7. Translation

In [ ]:
!macaw translate /tmp/test_audio.wav -m $STT_MODEL --server $BASE_URL

## 8. Cleanup

In [ ]:
!pkill -f 'macaw serve' || true
!echo 'Server stopped.'

In [ ]:
!macaw remove $STT_MODEL --yes

In [ ]:
!macaw remove $TTS_MODEL --yes

In [ ]:
!macaw list